# Build Topic Model
The present notebook will use the data collected in section 5.1 to compute and explore a topic model. In essence, a topic model consists of two sets of probability distributions. First, the probability that a *word* belongs to a certain topic (Topic/Term probability). Second, the probability of a *topic* to appear in a document (Document/Topic probability). These probabilities are never 0, that is, each word has some probability (even if very small) to be part of each topic and each topic has some probability (even if very small) to appear in a document.

A standard way to inspect a topic model is to look at the top-ten words of a topic (the ten words with the highest probability in each particular topic). Similarly, we can pull the ten most important documents for each topic (the documents in which this topic has the highest probability).

For a more thorough analysis we may create full probability tables: a topic/term probability table and a document/topic probability table. These tables give a fuller account of the model and will be used for the visualizations (5.3). 

In [ ]:
import numpy as np
import pandas as pd
from gensim import corpora, models, utils
import gensim

# Read in the texts
[For test purposes one may select only the first 100 documents. Remove the hashmark (#) from the first line of the following cell if you wish to do that]

In [ ]:
pickled = 'output/data_for_topic_model.p'
df = pd.read_pickle(pickled)
texts = df['lemma']

# POS-filter
The variable `posfilter` holds the last two characters of lemmatized words with allowed Part of Speech tags. If, for instance, you wish to select Verbs, Adjectives, and Nouns (in Akkadian), posfilter will be `[']n', 'aj', ']v']`. Note that one-character pos-tags need the right bracket!
The POS labels are:
* "n", #Nouns
* "v", #Verbs
* "aj", #Adjectives
* "av", #Adverbs
* "an", #Agricultural Name
* "cn", #Celestial Name
* "dn", #Divine Name
* "en", #Ethnicity Name
* "fn", #Field Name
* "gn", #Geographical Name (lands, etc.)
* "ln", #Lineage Name (ancestral clan)
* "mn", #Month Name
* "on", #Object Name
* "pn", #Personal Name
* "qn", #Quarter (of a city) Name
* "rn", #Royal Name
* "sn", #Settlement Name
* "tn", #Temple Name
* "wn", #Watercourse Name
* "yn", #Year Name
* "nu", #Numeral


In [ ]:
posfilter = [']n', ']v', 'aj']
#include nouns, verbs, and adjectives, not numerals, prepositions or proper nouns
texts = [[word for word in text if word[-2:] in posfilter] for text in texts]

# Stop words

Stop words are very frequent words that are not able to distinguish between topics. This includes, for instance, prepositions - but those can also be filtered out by the POS filter. The following nouns and verbs are too frequent to contribute to the analysis.

In [ ]:
stoplist = [
'šarru[king]n',
'bēlu[lord]n',
'libbu[interior]n',
'muhhu[skull]n',
'ardu[slave]n',
'šulmu[completeness]n',
'šapāru[send]v',
'alāku[go]v',
'qabû[say]v',
'pānu[front]n',
'māru[son]n',
'bītu[house]n',
'epēšu[do]v',
'wabālu[bring]v',
'šakānu[put]v',
'amāru[see]v',
'bašû[exist]v',
'našû[lift]v',
'izuzzu[stand]v',
'ūmu[day]n',
'ṭābu[good]aj',
'mādu[many]aj',
'nadānu[give]v',
'tadānu[give]v',
'ṣehru[small]aj',
'mimmû[all]n',
'gimru[totality]n',
'gabbu[totality]n',
'šâlu[ask]v',
'šemû[hear]v',
'ūmu[day]n',
'awātu[word]n',
'erēbu[enter]v'
]
texts = [[word for word in text if word not in stoplist] for text in texts]


# Filter out texts that have too few words left
Identify texts that have less than 10 lemmas left and use that selection for the list `texts` and for the dataframe `df`. 

In [ ]:
bo = [len(text)>9 for text in texts]
df = df[bo]
texts = [texts[i] for i in range(0, len(texts)) if bo[i]]

In [ ]:
len(bo), len(df)

How many documents did we start with, and how many do we have left?

# Dictionary
create the gensim Dictionary and filter for words that are too common or too rare (no_above may be set too low here).

In [ ]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=1, no_above=0.8)
## CHECK - is this done correctly?

In [ ]:
corpus = [dictionary.doc2bow(doc) for doc in texts]

# Compute the Model

Set the seed, indicate the number of topics (default set to 10) and run the model.

The visualization (section 5.3) will fail if the number of topics is higher than 25. 

In [ ]:
ntopics = int(input("Number of topics: ") or 10)
if ntopics > 25:
    ntopics = 25

In [ ]:
seed = 15
np.random.seed(seed)
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel
# Running and Training LDA model on the document term matrix.
ldamodel = Lda(corpus, num_topics=ntopics, id2word = dictionary, passes=50)

List the top 10 words and their probabilities in all topics. Note: the topic numbers here are not the ones used in the visualizations in 5.3! (The topics are the same, but not their numbers).

In [ ]:
ldamodel.show_topics(ntopics, formatted = False)

# Document/Topic Probability
The function `get_document_topics()` will list the probability of the topics in a single document. In order to get all the topics set the argument `minimum_probability` to zero. 

In [ ]:
ldamodel.get_document_topics(corpus[1], minimum_probability=0)

# Create Document/Topic Probability Table
A Document/Topiuc probability table is a table (DataFrame), where each row represents a document and each column a topuic. Each cell has the probability of a particular topic in a particular document. The sum of each row is 1 (probability distribution).

In order to create a full Document/Topic probability table we iterate over the entire corpus with the `get_document_topics()` function. This creates a list of lists (`list_of_doctopics`) where each list represents the probability of each topic in a document. The probability is represented in a tuple (topic_number, probability). The `list_of_probabilities` preserves only the probabilities. This list of lists is transformed into a DataFrame, whith as index the index of the original DataFrame with the tokenized data.  

In [ ]:
list_of_doctopics = [ldamodel.get_document_topics(corpus[i], minimum_probability=0) for i in range(len(corpus))]
list_of_probabilities = [[probability for label,probability in distribution] for distribution in list_of_doctopics]
d_t_df = pd.DataFrame(list_of_probabilities)
d_t_df = d_t_df.set_index(df.index)
d_t_df.head()

We can use the above table to find the ten highest scoring documents per topic with the pandas function 'nlargest'.

In [ ]:
new = pd.merge(df['designation'], d_t_df, left_index=True, right_index=True)
new

In [ ]:
doctop = []
for i in range(ntopics):
    t = new.nlargest(10, i)[['designation', i]]
    t['topic'] = i
    t = t.rename(columns = {i :'probability'})
    doctop.append(t)
doctop_df = pd.concat(doctop, axis=0)
doctop_df

# Renumber Topics
Rename the topics (columns) to start with 1, in accordance with the pyLDAvis visualization.

In [ ]:
topics = [i+1 for i in range(ntopics)]
d_t_df.columns = topics
d_t_df

# Create Topic / Term table
This is a table with N rows (the number of topics) and M columns (the number of individual terms in the Dictionary). The table indicates the probability of each term in each topic.

In [ ]:
topic_term = ldamodel.show_topics(ntopics, formatted=False, num_words=len(dictionary))

The object `topic_term` is a list of list. Each topic is represented by a list of tuples in the form `(word, probability)`. The following code pulls out the probabilities for each word in each topic (`topic_term[i][1]`) and creates a list of DataFrames with the words as index (rows) and the probabilities as the only column. The DataFrames are concatenated to a single DataFrame. 

In [ ]:
topic_term_list = [pd.DataFrame(topic_term[i][1]).set_index(0) for i in range(0, ntopics)]
t_t_df_ = pd.concat(topic_term_list, axis=1, ignore_index=True)
t_t_df_.head()

Rename the columns to start with 1, and Transpose to Topic/Term matrix.

In [ ]:
t_t_df_.columns = topics
t_t_df = t_t_df_.T
t_t_df

In [ ]:
#just checking
t_t_df['ēkallu[palace]n']

# TODO
Export data for viz. in 5.3